# Subclass BaseTool

You can define a custom tool by sub-classing from BaseTool. This provides maximal control over the tool definition, but requires writing more code.


#### guide:

https://python.langchain.com/v0.2/docs/how_to/custom_tools/#subclass-basetool

In [1]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.tools import StructuredTool


In [2]:
from typing import Optional, Type

from langchain.pydantic_v1 import BaseModel

from langchain_core.callbacks import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

from langchain_core.tools import BaseTool

In [3]:
class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


In [4]:
class addition(BaseTool):
    name = "Addition"
    description = "Return sum of 2 numbers"
    args_schema: Type[BaseModel] = CalculatorInput
    # return_direct: bool = True

    def _run(
        self, a: int, b: int, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return a + b

    async def _arun(
        self,
        a: int,
        b: int,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        # If the calculation is cheap, you can just delegate to the sync implementation
        # as shown below.
        # If the sync calculation is expensive, you should delete the entire _arun method.
        # LangChain will automatically provide a better implementation that will
        # kick off the task in a thread to make sure it doesn't block other async code.
        return self._run(a, b, run_manager=run_manager.get_sync())

In [5]:
class multiplication(BaseTool):
    name = "Multiply"
    description = "Return product of 2 numbers"
    args_schema: Type[BaseModel] = CalculatorInput
    # return_direct: bool = True

    def _run(
        self, a: int, b: int, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return a * b

    async def _arun(
        self,
        a: int,
        b: int,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        # If the calculation is cheap, you can just delegate to the sync implementation
        # as shown below.
        # If the sync calculation is expensive, you should delete the entire _arun method.
        # LangChain will automatically provide a better implementation that will
        # kick off the task in a thread to make sure it doesn't block other async code.
        return self._run(a, b, run_manager=run_manager.get_sync())

In [6]:
addition = addition()
multiplication= multiplication()


In [7]:
tools = [addition,multiplication]


In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

In [9]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

c:\Code\Github\LangChain-Basics\venv\lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [10]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [11]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [12]:
agent_executor.invoke({"input": "What is 3 +78 and what is 67*89"})



> Entering new AgentExecutor chain...

Invoking: `Addition` with `{'a': 3, 'b': 78}`


81
Invoking: `Multiply` with `{'a': 67, 'b': 89}`


5963The result of \(3 + 78\) is \(81\), and the result of \(67 \times 89\) is \(5963\).

> Finished chain.


{'input': 'What is 3 +78 and what is 67*89',
 'output': 'The result of \\(3 + 78\\) is \\(81\\), and the result of \\(67 \\times 89\\) is \\(5963\\).'}

In [13]:
agent_executor.invoke({"input": "if i have 3 apples and i get 567 more apples. How many apples will I have in all?"})



> Entering new AgentExecutor chain...

Invoking: `Addition` with `{'a': 3, 'b': 567}`


570If you have 3 apples and you get 567 more, you will have a total of 570 apples.

> Finished chain.


{'input': 'if i have 3 apples and i get 567 more apples. How many apples will I have in all?',
 'output': 'If you have 3 apples and you get 567 more, you will have a total of 570 apples.'}